In [20]:
from atgreen import *
from tqdm import tqdm
import re

In [ ]:
PATH=""

In [2]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle', 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']

In [ ]:
# Generate indexes on table with distances
engine=create_engine(f"postgresql+psycopg2://{db_params['db_user']}:{db_params['db_password']}@{db_params['db_host']}:5432/{db_params['db_name']}")       

sql="""SELECT * FROM information_schema.tables
       WHERE table_schema = 'distances' """
tables_list=list(pd.read_sql(sql, engine)["table_name"])


for k,city in tqdm(eunmerate(cities_list)):
    if city in tables_list:
        generate_indexes4table(f"idx_{k[k.rfind('_')+1:]}_x", 'distances', city, "x_source", db_params)
        generate_indexes4table(f"idx_{k[k.rfind('_')+1:]}_y", 'distances', city, "y_source",db_params)
        generate_indexes4table(f"idx_{k[k.rfind('_')+1:]}_wd", 'distances', city, "walk_minutes",db_params)

In [ ]:
# Generate indexes on table for remapped green 
generate_indexes4table("cell_index_x", "osm", "osm2grid", "x_source", db_params)
generate_indexes4table("cell_index_y", "osm", "osm2grid", "y_source",db_params)
for ind in range(7):
    generate_indexes4table(f"_gs_index_{ind}", "osm", "osm2grid", f"{ind}_gs",db_params)
generate_indexes4table("cell_index_x", "esa", "esa2grid", "x_source", db_params)
generate_indexes4table("cell_index_y", "esa", "esa2grid", "y_source",db_params)
for ind in range(1):
    generate_indexes4table(f"_gs_index_{ind}", "esa", "esa2grid", f"{ind}_gs",db_params)